Goal of analysis script is to pull in excel files with manually entered intake and weight data, determine daily intake per body weight (for both water and ethanol) for subsequent analysis


Import numpy

In [60]:
import pandas as pd
import re

In [61]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

Open dialog box and prompt user to select file containing the data

In [62]:
from tkinter import filedialog
from tkinter import *
 
root = Tk()
root.filename =  filedialog.askopenfilename(initialdir = "/",title = "Select file",filetypes = (("jpeg files","*.jpg"),("all files","*.*")))
print (root.filename)

C:/Users/Schindler/Documents/Schindler_Lab/Data/Ethanol/Intermit 2BC/Round 2/second org/386.xlsx


Generate pandas dataframe from excel file

Get animal number from path name (animal number is file name), create array of animal number the length of the data_table so can add this as a column

Add animal number array as a column to data table. Add empty columns for the intake and preference data we will compute in the next step.

In [65]:
data = pd.read_excel(root.filename)
data_table = pd.DataFrame(data = data)

name = root.filename
name = re.search("\d\d\d", name).group(0)
name = [name] * len(data_table)

data_table = data_table.assign(Animal = name)
data_table.head()

,Day,Day #,EtOH,Weekend,Notes,Weight,EtOh position,EtOH bf,EtOH after,H2O rt b4,H2O rt after,H2O lt b4,H2O lt after,EtOH 8,H20 rt 8,H20 lt 8,Animal
0,Friday0,1,0,1,NaN,32.4,X,x,x,89.3,86.3,86.6,83.5,x,x,x,386
1,Sunday 3%,2,3,0,NaN,32.4,R,56,54.2,x,x,83.5,81.7,x,x,x,386
2,Monday1,3,0,0,NaN,32.4,X,x,x,86.1,85,81.7,78,x,x,x,386
3,Tuesday 6%,4,6,0,NaN,32.4,L,58,55,85,84.7,x,x,x,x,x,386
4,Wednesday1,5,0,0,NaN,32.4,X,x,x,84.7,84,78.6,75.6,x,x,x,386


In [67]:
data_table['H2O_intake_rt_raw'] = \
data_table['H2O rt b4'].where(data_table['EtOh position'] != 'R') \
- data_table['H2O rt after'].where(data_table['EtOh position'] != 'R')

data_table['H2O_intake_lt_raw'] = \
data_table['H2O lt b4'].where(data_table['EtOh position'] != 'L') \
- data_table['H2O lt after'].where(data_table['EtOh position'] != 'L')

data_table['H2O_intake_lt'] = \
data_table['H2O_intake_lt_raw'] / (data_table['Weight'] /1000)

data_table['H2O_intake_rt'] = \
data_table['H2O_intake_rt_raw'] / (data_table['Weight'] /1000)

data_table['H2O_pref'] = \
data_table['H2O_intake_rt_raw'] / (data_table['H2O_intake_rt_raw'] + data_table['H2O_intake_lt_raw'])

data_table['H2O_intake_rt_raw_8'] = \
data_table['H2O rt b4'].where(data_table['H20 rt 8'] != 'x') \
- data_table['H20 rt 8'].where(data_table['H20 rt 8'] != 'x')

data_table['H2O_intake_lt_raw_8'] = \
data_table['H2O lt b4'].where(data_table['H20 lt 8'] != 'x') \
- data_table['H20 lt 8'].where(data_table['H20 lt 8'] != 'x')

data_table['H2O_intake_lt_8'] = \
data_table['H2O_intake_lt_raw_8'] / (data_table['Weight'] /1000)

data_table['H2O_intake_rt_8'] = \
data_table['H2O_intake_rt_raw_8'] / (data_table['Weight'] /1000)

data_table['EtOH_intake_raw'] = \
data_table['EtOH bf'].where(data_table['EtOH bf'] != 'x') - data_table['EtOH after'].where(data_table['EtOH bf'] != 'x')

data_table['EtOH_intake'] = \
(data_table['EtOH_intake_raw'] * 0.2) / (data_table['Weight'] /1000)

data_table['EtOH_intake_raw_8'] = \
data_table['EtOH bf'].where(data_table['EtOH 8'] != 'x') - data_table['EtOH 8'].where(data_table['EtOH 8'] != 'x')

data_table['EtOH_intake_8'] = \
(data_table['EtOH_intake_raw_8'] * 0.2) / (data_table['Weight'] /1000)



data_table = data_table.fillna(value = 0)

data_table['H2O_intake_raw'] = \
data_table['H2O_intake_rt_raw'] + data_table['H2O_intake_lt_raw']

data_table['H2O_intake'] = \
data_table['H2O_intake_rt'] + data_table['H2O_intake_lt']

data_table['H2O_intake_raw_8'] = \
data_table['H2O_intake_rt_raw_8'] + data_table['H2O_intake_lt_raw_8']

data_table['H2O_intake_8'] = \
data_table['H2O_intake_rt_8'] + data_table['H2O_intake_lt_8']

data_table['EtOH_pref'] = \
data_table['EtOH_intake_raw'] / (data_table['EtOH_intake_raw'] + data_table['H2O_intake_raw'])

data_table['EtOH_pref_8'] = \
data_table['EtOH_intake_raw_8'] / (data_table['EtOH_intake_raw_8'] + data_table['H2O_intake_raw_8'])


data_table.head()

,Day,Day #,EtOH,Weekend,Notes,Weight,EtOh position,EtOH bf,EtOH after,H2O rt b4,H2O rt after,H2O lt b4,H2O lt after,EtOH 8,H20 rt 8,H20 lt 8,Animal,H2O_intake_rt_raw,H2O_intake_lt_raw,H2O_intake_lt,H2O_intake_rt,H2O_pref,H2O_intake_rt_raw_8,H2O_intake_lt_raw_8,H2O_intake_lt_8,H2O_intake_rt_8,EtOH_intake_raw,EtOH_intake,EtOH_intake_raw_8,EtOH_intake_8,H2O_intake_raw,H2O_intake,H2O_intake_raw_8,H2O_intake_8,EtOH_pref,EtOH_pref_8
0,Friday0,1,0,1,0.0,32.4,X,x,x,89.3,86.3,86.6,83.5,x,x,x,386,3.0,3.1,95.679012,92.592593,0.491803,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,6.1,188.271605,0.0,0.0,0.000000,NaN
1,Sunday 3%,2,3,0,0.0,32.4,R,56,54.2,x,x,83.5,81.7,x,x,x,386,0.0,1.8,55.555556,0.000000,0.000000,0.0,0.0,0.0,0.0,1.8,11.111111,0.0,0.0,1.8,55.555556,0.0,0.0,0.500000,NaN
2,Monday1,3,0,0,0.0,32.4,X,x,x,86.1,85,81.7,78,x,x,x,386,1.1,3.7,114.197531,33.950617,0.229167,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,4.8,148.148148,0.0,0.0,0.000000,NaN
3,Tuesday 6%,4,6,0,0.0,32.4,L,58,55,85,84.7,x,x,x,x,x,386,0.3,0.0,0.000000,9.259259,0.000000,0.0,0.0,0.0,0.0,3.0,18.518519,0.0,0.0,0.3,9.259259,0.0,0.0,0.909091,NaN
4,Wednesday1,5,0,0,0.0,32.4,X,x,x,84.7,84,78.6,75.6,x,x,x,386,0.7,3.0,92.592593,21.604938,0.189189,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,3.7,114.197531,0.0,0.0,0.000000,NaN


In [58]:
data_table_all_calc = data_table.drop(['Notes', 'Weight', 'EtOh position', 'EtOH bf', 'EtOH after', 'H2O rt b4', 'H2O rt after', 'H2O lt b4', 'H2O lt after', 'EtOH 8', 'H20 rt 8', 'H20 lt 8'], axis=1)
data_table_all_calc.head()

,Day,Day #,EtOH,Weekend,Animal,H2O_intake_rt_raw,H2O_intake_lt_raw,H2O_intake_lt,H2O_intake_rt,H2O_pref,H2O_intake_rt_raw_8,H2O_intake_lt_raw_8,H2O_intake_lt_8,H2O_intake_rt_8,EtOH_intake_raw,EtOH_intake,EtOH_intake_raw_8,EtOH_intake_8,H2O_intake_raw,H2O_intake,H2O_intake_raw_8,H2O_intake_8,EtOH_pref,EtOH_pref_8
0,Friday0,1,0,1,387,0.6,4.5,135.379061,18.050542,0.117647,0,0.0,0.0,0,0.0,0.000000,0.0,0.0,5.1,153.429603,0.0,0.0,0.000000,NaN
1,Sunday 3%,2,3,0,387,0.0,1.9,57.160048,0.000000,0.000000,0,0.0,0.0,0,1.8,10.830325,0.0,0.0,1.9,57.160048,0.0,0.0,0.486486,NaN
2,Monday1,3,0,0,387,2.0,1.5,45.126354,60.168472,0.571429,0,0.0,0.0,0,0.0,0.000000,0.0,0.0,3.5,105.294826,0.0,0.0,0.000000,NaN
3,Tuesday 6%,4,6,0,387,0.6,0.0,0.000000,18.050542,0.000000,0,0.0,0.0,0,2.8,16.847172,0.0,0.0,0.6,18.050542,0.0,0.0,0.823529,NaN
4,Wednesday1,5,0,0,387,1.0,2.7,81.227437,30.084236,0.270270,0,0.0,0.0,0,0.0,0.000000,0.0,0.0,3.7,111.311673,0.0,0.0,0.000000,NaN


In [59]:
negatives_bool = data_table_all_calc < 0.0
negatives = data_table_all_calc[negatives_bool]
negatives

,Day,Day #,EtOH,Weekend,Animal,H2O_intake_rt_raw,H2O_intake_lt_raw,H2O_intake_lt,H2O_intake_rt,H2O_pref,H2O_intake_rt_raw_8,H2O_intake_lt_raw_8,H2O_intake_lt_8,H2O_intake_rt_8,EtOH_intake_raw,EtOH_intake,EtOH_intake_raw_8,EtOH_intake_8,H2O_intake_raw,H2O_intake,H2O_intake_raw_8,H2O_intake_8,EtOH_pref,EtOH_pref_8
0,Friday0,NaN,NaN,NaN,387,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Sunday 3%,NaN,NaN,NaN,387,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Monday1,NaN,NaN,NaN,387,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Tuesday 6%,NaN,NaN,NaN,387,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Wednesday1,NaN,NaN,NaN,387,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Thursday 9%,NaN,NaN,NaN,387,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Friday1,NaN,NaN,NaN,387,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Sunday 20%,NaN,NaN,NaN,387,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Monday2,NaN,NaN,NaN,387,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Tuesday 20%,NaN,NaN,NaN,387,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
def I2BC(data):
    "Analyzes intermittent two bottle choice data"
    
    data_table['H2O_intake_rt_raw'] = \
    data_table['H2O rt b4'].where(data_table['EtOh position'] != 'R') \
    - data_table['H2O rt after'].where(data_table['EtOh position'] != 'R')

    data_table['H2O_intake_lt_raw'] = \
    data_table['H2O lt b4'].where(data_table['EtOh position'] != 'L') \
    - data_table['H2O lt after'].where(data_table['EtOh position'] != 'L')

    data_table['H2O_intake_lt'] = \
    data_table['H2O_intake_lt_raw'] / (data_table['Weight'] /1000)

    data_table['H2O_intake_rt'] = \
    data_table['H2O_intake_rt_raw'] / (data_table['Weight'] /1000)

    data_table['H2O_pref'] = \
    data_table['H2O_intake_rt_raw'] / (data_table['H2O_intake_rt_raw'] + data_table['H2O_intake_lt_raw'])

    data_table = data_table.fillna(value = 0)

    data_table['H2O_intake_raw'] = \
    data_table['H2O_intake_rt_raw'] + data_table['H2O_intake_lt_raw']

    data_table['H2O_intake'] = \
    data_table['H2O_intake_rt'] + data_table['H2O_intake_lt']
    
    
    return data

Use the intake function on the data table

In [18]:
intake_data = data_table.pipe(I2BC)
intake_data.head()

,Day,Weight,EtOH position,EtOH wt,H2O rt wt,H2O lt wt,EtOH wt.1,H2O rt wt.1,H2O lt wt.1,Animal,EtOH,EtOH8,EtOH8_pref,EtOH_pref,H2O,H2O8,H2O_pref,H20,H20_pref,H208
0,Friday,30,X,0,70,70,X,X,X,111,NaN,NaN,NaN,NaN,,,,333.333333,1.0,NaN
1,Sunday 3%,30,R,60,65,65,X,X,X,111,6.666667,NaN,NaN,0.1,,,,66.666667,NaN,NaN
2,Monday,30,X,59,63,63,X,X,X,111,NaN,NaN,NaN,NaN,,,,133.333333,1.0,NaN
3,Tuesday 6%,30,L,59,61,61,X,X,X,111,13.333333,NaN,NaN,0.4,,,,33.333333,NaN,NaN
4,Wednesday,30,X,57,60,60,X,X,X,111,NaN,NaN,NaN,NaN,,,,66.666667,1.0,NaN
